In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F

from IPython.display import display, HTML
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from robotpal import Robot, Camera, bgr8_to_jpeg
from robotpal.SCSCtrl import TTLServo
from robotpal._core.server import SimulatorServer

import threading
import time
import cv2
import PIL.Image
import numpy as np
from vision.detector import PlateNumberDetector
import threading
import time

display(HTML("<style>.big-log-text pre { font-size: 30px !important; line-height: 1.4 !important; font-weight: bold; }</style>"))
log_output = widgets.Output(layout={'border': '1px solid black', 'height': '300px', 'overflow_y': 'scroll'})
log_output.add_class("big-log-text")

ocr_detector = PlateNumberDetector(
    model="paddle",
    plate_similarity_thresh=80,
    debug_mode=False  # 실전 모드 (카메라 영상 분석)
)
print("OCR Detector Ready!")

[RobotPal] 서버 시작됨 | WebSocket: 9999, TCP: 9998
[RobotPal] 통신 대기 중... (WS: 9999, TCP: 9998)
[System] WebSocket 프로세서 시작


Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\korean_PP-OCRv5_mobile_rec`.


OCR Detector Ready!


In [2]:
robot = Robot()
camera = Camera()
# camera_ocr = Camera.instance(width=816, height=616)
TTLServo.servoAngleCtrl(5, 45, 1, 100)

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

# .pth 파일의 경로를 확인
model.load_state_dict(torch.load('best_steering_model_xy_test_12_17.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
print('model load success')

lbl1 = ipywidgets.Label(value="areaA :")
areaAlbl = ipywidgets.Label(value="None")
hbox1 = widgets.HBox([lbl1, areaAlbl] )

lbl2 = ipywidgets.Label(value="areaB :")
areaBlbl = ipywidgets.Label(value="None")
hbox2 = widgets.HBox([lbl2,areaBlbl])

lbl3 = ipywidgets.Label(value="self.flag :")
flaglbl = ipywidgets.Label(value="None")
hbox3 = widgets.HBox([lbl3, flaglbl])
vbox1 = widgets.VBox([hbox1, hbox2, hbox3])

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
vbox2 = widgets.VBox([image_widget, x_slider, steering_slider], layout=widgets.Layout(align_self='center'))
hbox4 = widgets.HBox([vbox2, y_slider, speed_slider], layout=widgets.Layout(align_self='center'))

startBtn = widgets.Button(description="Start", button_style='info')
lbl41 = ipywidgets.Label(value="Find Area : ")
goallbl = ipywidgets.Label(value="None")
hbox5 = widgets.HBox([startBtn,lbl41, goallbl])


#수동 조작용 contorller widget 생성
lbl50 = ipywidgets.Label(value="Manual Controller")

button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([lbl50, forward_button, middle_box, backward_button])

#자동 조작용 contorller widget 생성
lbl51 = ipywidgets.Label(value="Auto Controller")
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.25, description='speed gain',disabled = False)
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain',disabled = False)
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.5, description='steering kd',disabled = False)
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias',disabled = False)
vbox3 = widgets.VBox([lbl51,speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])
hbox6 = widgets.HBox([controls_box, vbox3], layout=widgets.Layout(align_self='center'))
log_output = ipywidgets.Output(layout={'border': '1px solid black', 'height': '800px', 'overflow_y': 'scroll'})

display(vbox1, hbox4,hbox5,hbox6, log_output)

def print_log(message):
    log_output.append_stdout(message + "\n")

manual_btnlst = [stop_button, forward_button, backward_button, left_button, right_button]
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)

def step_backward(change):
    robot.backward(0.4)

def step_left(change):
    robot.left(0.3)
    time.sleep(0.5)
    robot.stop()

def step_right(change):
    robot.right(0.3)
    time.sleep(0.5)
    robot.stop()

stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)

# areaA 와 areaB 를 선정
areaA = 'mint'
areaB = 'darkblue'
# 조사한 색상의 이름과 min, max HSV 값, 그리고 회전 각도(angle)를 담은 리스트
# [수정] colors 리스트 (여러 타겟을 리스트 []로 묶음)
colors = [
    # targets: 해당 영역에서 찾아야 할 차량 번호들 (리스트 형태)
    {'name': 'red',      'lower': np.array([3, 217, 214]),   'upper': np.array([4, 222, 218]),    'angle': 0,     'targets': [] },
    {'name': 'mint',     'lower': np.array([68, 17, 221]),   'upper': np.array([70, 20, 222]),    'angle': 32.70, 'targets': ['263주0300', '255도7345'] }, 
    {'name': 'blue',     'lower': np.array([106, 212, 207]), 'upper': np.array([108, 223, 216]),  'angle': 29.47, 'targets': [] }, 
    {'name': 'darkblue', 'lower': np.array([120, 201, 182]), 'upper': np.array([120, 206, 189]),  'angle': 29.57, 'targets': ['187고1604', '249마1706'] }, 
    {'name': 'yellow',   'lower': np.array([27, 85, 195]),   'upper': np.array([30, 97, 221]),    'angle': 0,     'targets': [] },
    {'name': 'orange',   'lower': np.array([17, 145, 222]),  'upper': np.array([18, 154, 224]),   'angle': 0,     'targets': [] }
]

areaA_color = next((color for color in colors if color['name'] == areaA), None)
areaB_color = next((color for color in colors if color['name'] == areaB), None)

areaAlbl.value = areaA_color['name']
areaBlbl.value = areaB_color['name']

findArea = areaA
goallbl.value = findArea

#frame 크기와 카메라 중심점 좌표 설정
frame_width = 224
frame_height = 224
camera_center_X = int(frame_width/2)
camera_center_Y = int(frame_height/2)

#WorkingAreaFind() 용 리스트와 변수
colorHSVvalueList = []
max_len = 20

#2개 thread 용 객체 변수 생성
roadFinding = None
goalFinding = None

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


model load success


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
import threading
import time

# 쓰레드 간 통신을 위한 이벤트 객체
ocr_trigger_event = threading.Event()  # "OCR 시작해!" 신호
ocr_done_event = threading.Event()     # "OCR 다 했어!" 신호

# 데이터를 주고받을 공유 변수
shared_ocr_image = None   # OCR 할 이미지 담을 곳
shared_ocr_result = None  # 결과 담을 곳
shared_target_list = []   # 찾아야 할 번호판 목록

class OCRWorker(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True # 메인 프로그램 꺼지면 같이 꺼짐
        self.running = True
        
    def run(self):
        global shared_ocr_image, shared_ocr_result
        
        while self.running:
            # 1. 일이 들어올 때까지 대기 (CPU 안 씀)
            ocr_trigger_event.wait()
            
            # 이벤트가 꺼져있으면 다시 대기 (중복 방지)
            if not ocr_trigger_event.is_set():
                continue

            print_log(">>> OCR 연산 시작...")
            
            # 2. OCR 수행
            if shared_ocr_image is not None:
                try:

                    h, w, c = shared_ocr_image.shape
                    print_log(f">>> [Check] Input Image Size: {w}x{h}")
                    
                    # 2. 이미지 파일로 저장 (현재 폴더에 저장됨)
                    filename = f"debug_ocr_input_{int(time.time())}.jpg"
                    cv2.imwrite(filename, shared_ocr_image)
                    print_log(f">>> [Saved] {filename} 저장됨! 확인해보세요.")

                    # 3. OCR 실행
                    results = ocr_detector.detect(shared_ocr_image, target='255도7345')
                    shared_ocr_result = results
                except Exception as e:
                    print_log(f"OCR Error: {e}")
                    shared_ocr_result = None
            else:
                print_log("OCR Image is None")
                shared_ocr_result = None

            # print_log("<<< [Thread 3] OCR 연산 완료!")

            # 3. 완료 신호 보내기
            ocr_done_event.set()      # "다 했어" 깃발 들기
            ocr_trigger_event.clear() # "시작해" 깃발 내리기


class WorkingAreaFind(threading.Thread):
    def __init__(self):
        super().__init__()
        self.th_flag = True
        self.imageInput = 0
        self.flag = 1
        flaglbl.value = str(self.flag)
        
        # [중요] OCR 상태 변수 초기화
        self.ocr_running = False 
        self.ocr_results = [] 

    def wait_and_show(self, seconds, message=""):
        if message:
            print_log(f"▶ {message}")
            
        start_time = time.time()
        while time.time() - start_time < seconds:
            self.imageInput = camera.value
            if message:
                cv2.putText(self.imageInput, message, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            image_widget.value = bgr8_to_jpeg(self.imageInput)
            time.sleep(0.05)

    def run(self):
        while self.th_flag:
            self.imageInput = camera.value
            hsv = cv2.cvtColor(self.imageInput, cv2.COLOR_BGR2HSV)
            hsv = cv2.blur(hsv, (15, 15))
                        
            areaA_mask = cv2.inRange(hsv, areaA_color['lower'], areaA_color['upper'])
            areaA_mask = cv2.erode(areaA_mask, None, iterations=2)
            areaA_mask = cv2.dilate(areaA_mask, None, iterations=2)
            
            areaB_mask = cv2.inRange(hsv, areaB_color['lower'], areaB_color['upper'])
            areaB_mask = cv2.erode(areaB_mask, None, iterations=2)
            areaB_mask = cv2.dilate(areaB_mask, None, iterations=2)

            AContours, _ = cv2.findContours(areaA_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            BContours, _ = cv2.findContours(areaB_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if AContours and self.flag == 1:
                self.findCenter(areaA, AContours)
            elif BContours and self.flag == 2:
                self.findCenter(areaB, BContours)
            else:
                cv2.putText(self.imageInput, "Finding...", (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
                image_widget.value = bgr8_to_jpeg(self.imageInput)
            time.sleep(0.1)
    
    def findCenter(self, name, Contours):  
        global roadFinding, findArea, shared_ocr_image, shared_ocr_result

        c = max(Contours, key=cv2.contourArea)
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)

        X = int(box_x)
        Y = int(box_y)
        
        error_Y = abs(camera_center_Y - Y)
        error_X = abs(camera_center_X - X)
        
        if error_Y < 15 and error_X < 15:
            is_valid_target = False
            if name == areaA and self.flag == 1:
                is_valid_target = True
            elif name == areaB and self.flag == 2:
                is_valid_target = True
            
            if is_valid_target:
                self.wait_and_show(1.0, "Approaching Goal...")

                if 'roadFinding' in globals() and roadFinding:
                    roadFinding.paused = True
                robot.stop()

                if name == areaA:
                    self.flag = 2
                    findArea = areaB
                    goallbl.value = findArea
                    areaAlbl.value = areaA + " Goal!"
                    flaglbl.value = str(self.flag)
                elif name == areaB:              
                    self.flag = 1       
                    findArea = areaB
                    goallbl.value = findArea
                    areaBlbl.value = areaB + " Goal!"
                    flaglbl.value = str(self.flag)   

                current_color_info = next((c for c in colors if c['name'] == name), None)
                target_angle = current_color_info['angle'] if current_color_info else 0
                target_list = current_color_info.get('targets', []) if current_color_info else []

                if target_angle != 0:
                    # 1. 준비 동작
                    TTLServo.servoAngleCtrl(1, target_angle, 1, 100)
                    self.wait_and_show(1.5, "Adjusting Camera...")

                    # 2. 이미지 준비 (고화질)
                    try: 
                        # 1. 서버 인스턴스 소환 (싱글톤이라 현재 실행 중인 서버가 잡힘)
                        server = SimulatorServer.instance()
                        
                        # 2. 원본 데이터 가져오기
                        raw_bytes = server.latest_jpeg
                        
                        if raw_bytes is not None:
                            # 2. 바이트 -> 이미지 배열로 변환 (디코딩)
                            nparr = np.frombuffer(raw_bytes, np.uint8)
                            decoded_img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                            
                            # 3. 로봇 카메라는 뒤집혀 있는 경우가 많으므로 상하반전 (Flip)
                            shared_ocr_image = cv2.flip(decoded_img, 0)
                            
                            h, w, _ = shared_ocr_image.shape
                            print_log(f"▶ 고화질 원본 추출 성공! ({w}x{h})")
                        else:
                            # 데이터가 없으면 어쩔 수 없이 저화질 사용
                            shared_ocr_image = camera.value.copy()
                            print_log("▶ [경고] 원본 데이터(latest_jpeg) 없음 -> 저화질 사용")
                            
                    except Exception as e: 
                        shared_ocr_image = None
                        print_log(f"▶ 이미지 디코딩 실패: {e}")

                    # 3. [핵심] Thread 3에게 일 시키기
                    ocr_done_event.clear()    # 완료 깃발 내림
                    ocr_trigger_event.set()   # "시작해" 깃발 올림

                    # 4. 완료될 때까지 "기다리면서 화면 갱신" (이게 3-Thread의 장점)
                    # OCR이 도는 동안 여기 while문이 계속 돌면서 화면을 그려줍니다.
                    start_wait = time.time()
                    while not ocr_done_event.is_set():
                        self.imageInput = camera.value
                        
                        # 점 찍기 애니메이션 (Waiting...)
                        elapsed = int(time.time() * 2) % 4
                        dots = "." * elapsed
                        cv2.putText(self.imageInput, f"OCR Processing{dots}", (10, 100), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
                        
                        image_widget.value = bgr8_to_jpeg(self.imageInput)
                        time.sleep(0.05)
                        
                        # 10초 넘으면 강제 탈출 (안전장치)
                        if time.time() - start_wait > 30:
                            print_log("OCR Timeout!")
                            break

                    # 5. 결과 확인
                    results = shared_ocr_result
                    if results:
                        detected_texts = []
                        found_match = False
                        
                        # 리스트 안전 처리
                        if not isinstance(results, list): results = [results]
                        
                        for res in results:
                            if hasattr(res, 'text'):
                                txt = res.text.replace(" ", "").strip()
                                detected_texts.append(txt)
                                for t in target_list:
                                    if t.replace(" ", "").strip() == txt:
                                        found_match = True
                        
                        print_log(f"--- 결과: {detected_texts}")
                        msg = "MATCH FOUND!" if found_match else "No Match"
                        self.wait_and_show(3.0, msg)
                    else:
                        self.wait_and_show(2.0, "No Text Detected")

                    # 6. 복귀
                    TTLServo.servoAngleCtrl(1, 0, 1, 100)
                    self.wait_and_show(1.5, "Returning...")

                else:
                    self.wait_and_show(5.0, "Frontal View...")

                if 'roadFinding' in globals() and roadFinding:
                    roadFinding.paused = False
                    
        image_widget.value = bgr8_to_jpeg(self.imageInput)
        
    def stop(self):
        self.th_flag = False
        robot.stop()

class RobotMoving(threading.Thread):
    def __init__(self):
        super().__init__()
        self.th_flag = True
        self.paused = False
        self.angle = 0.0
        self.angle_last = 0.0
        
    def run(self):
        while self.th_flag:

            if self.paused:
                robot.stop()
                time.sleep(0.1)
                continue


            image = camera.value
            xy = model(self.preprocess(image)).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0
            
            x_slider.value = x
            y_slider.value = y
            
            #인공지능 무인운반로봇(AGV)의 속도 표시
            speed_slider.value = 0.23
            
            image_widget.value = bgr8_to_jpeg(image)
            
            #조향값 계산
            self.angle = np.arctan2(x, y)
            
            if not self.th_flag:
                break
            #PID 제어를 이용한 모터 제어
            pid = self.angle * 0.2 + (self.angle - self.angle_last) * 0.5
            self.angle_last = self.angle

            #슬라이더에 표시
            steering_slider.value = pid

            robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), -0.9)
            robot.right_motor.value =  max(min(speed_slider.value - steering_slider.value, 1.0), -0.9)
            time.sleep(0.1)
        robot.stop()
    
    def preprocess(self, image):
        image = PIL.Image.fromarray(image)
        image = transforms.functional.to_tensor(image).to(device).half()
        image.sub_(mean[:, None, None]).div_(std[:, None, None])
        return image[None, ...]

    def stop(self):
        self.th_flag = False
        robot.stop()

def start(change):
    global camera_link, goalFinding, roadFinding, ocr_thread
    
    if startBtn.description == "Start" :
        print_log("System Started! Target Finding Begin...")
        camera_link.unlink()
        
        # [핵심] OCR 쓰레드를 여기서 반드시 생성하고 시작해야 합니다!
        ocr_thread = OCRWorker()
        ocr_thread.start()
        
        goalFinding = WorkingAreaFind()
        goalFinding.start()
        
        roadFinding = RobotMoving()
        roadFinding.start()

        startBtn.button_style = "warning" 
        startBtn.description = "Stop"
    
    elif startBtn.description == "Stop":
        if roadFinding: roadFinding.stop()
        if goalFinding: goalFinding.stop()
        
        # OCR 쓰레드 종료
        if ocr_thread: 
            ocr_thread.running = False
            # 대기 중인 쓰레드를 깨워서 종료시킴
            ocr_trigger_event.set()
        
        camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
            
        startBtn.button_style = "info"
        startBtn.description = "Start"

startBtn.on_click(start)

In [ ]:
time.sleep(0.1)
robot.stop()
camera.stop()

print('End')